<a href="https://colab.research.google.com/github/KrishnaMurali177/Deep-Learning-Terrain-Identifiaction/blob/main/Terrain_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import collections
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM, LSTM, SimpleRNN, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np

In [3]:
import glob
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
all_filenames = [i for i in glob.glob('/content/drive/My Drive/TrainingData/*.{}'.format('csv'))]

In [23]:
df_x, df_x_time, df_y, df_y_time = [], [], [], []
for f in all_filenames:
    if f.endswith('x.csv'):
      df = pd.read_csv(f, header=None)
      df['subject'] = f.split('/')[-1].split('_')[1]
      df['subject_ID'] = f.split('/')[-1].split('_')[2]
      df_x.append(df)
    if f.endswith('x_time.csv'):
      df = pd.read_csv(f, header=None)
      df_x_time.append(df)
    if f.endswith('y.csv'):
      df = pd.read_csv(f, header=None)
      df['subject'] = f.split('/')[-1].split('_')[1]
      df['subject_ID'] = f.split('/')[-1].split('_')[2]
      df_y.append(df)
    if f.endswith('y_time.csv'):
      df = pd.read_csv(f, header=None)
      df_y_time.append(df)
    
x = pd.concat(df_x, ignore_index=True)
x.columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'subject', 'subject_ID']
x_time = pd.concat(df_x_time, ignore_index=True)
x_time.columns = ['x_time']
y = pd.concat(df_y, ignore_index=True)
y.columns = ['y', 'subject', 'subject_ID']
y_time = pd.concat(df_y_time, ignore_index=True)
y_time.columns = ['y_time']
x['x_time'] = x_time['x_time']
y['y_time'] = y_time['y_time']

In [24]:
x.head()

,x1,x2,x3,x4,x5,x6,subject,subject_ID,x_time
0,4.4353,8.1961,2.9745,0.014215,-0.039157,-0.016744,001,01,0.000
1,4.1869,8.3445,2.9081,0.005771,-0.004479,-0.003345,001,01,0.025
2,4.5446,8.4087,2.8900,0.007967,0.022412,0.001159,001,01,0.050
3,4.8493,8.4116,2.9007,0.027778,-0.010670,-0.014223,001,01,0.075
4,4.5092,8.1186,2.8473,0.021577,-0.045498,-0.021111,001,01,0.100


In [34]:
y.groupby('y').nunique()

,subject,subject_ID,y_time
y,,,
0,8,8,17230
1,8,7,7198
2,8,8,10562
3,8,8,12501
